In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.signal
from utils import encode, ofdm, channels, audio, decode,preamble
import sounddevice as sd
cwd = os.getcwd()

In [ ]:
DFT_LENGTH = 4096
CP_LENGTH = 512
silence_duration = 1
chirp_duration = 1
fs = 48000

low_freq = 1000
high_freq = 10000
double_chirp = False

encode_method = 'qpsk'
if encode_method == 'bpsk':
    bits_per_symbol = 1
elif encode_method == 'qpsk':
    bits_per_symbol = 2

In [ ]:
# Generate a bit array as data to transmit
new = True

spb = ofdm.subcarriers_per_block(fs,DFT_LENGTH,low_freq,high_freq)

if new:
    np.random.seed(8)
    encoded = np.random.randint(2, size=2 * spb * 128)
    np.savetxt('encoded', encoded, fmt='%i')

# Load a saved bit array
encoded = np.loadtxt('encoded', dtype=np.int32)

In [ ]:
if encode_method == 'qpsk':
    symbols = encode.qpsk_encode(encoded)
elif encode_method == 'bpsk':
    symbols = encode.bpsk_encode(encoded)

shifted = ofdm.subcarrier_shift_gaussian(symbols, DFT_LENGTH, fs, low_freq, high_freq, 0.01, bits_per_symbol, constellation=encode_method)
ofdm_data = ofdm.symbols_to_ofdm(shifted, DFT_LENGTH, CP_LENGTH)

In [ ]:
#generate known ofdm symbols
known_ofdm = True
seed_known = 7
repeat_time = 4
if known_ofdm:
    known_ofdm_data = preamble.generate_known_ofdm(fs,DFT_LENGTH,CP_LENGTH,low_freq,high_freq,encode_method,repeat_time,seed_known)
else:
    known_ofdm_data = np.array([])

In [ ]:
# Generate preamble and assemble
chirp_start = preamble.generate_chirp(chirp_duration, fs, low=low_freq, high=high_freq, silence_duration=silence_duration, double=double_chirp)
# add chirp to end of transmission
chirp_end = preamble.generate_chirp(chirp_duration, fs, low=low_freq, high=high_freq, silence_duration=0, double=double_chirp)

#assemble
#output_audio = np.concatenate((chirp_start,np.real(known_ofdm_data),np.real(ofdm_data),chirp_end,np.zeros(fs*silence_duration)))


output_audio = np.concatenate((chirp_start,15*np.real(known_ofdm_data),15*np.real(ofdm_data),15*np.real(known_ofdm_data),chirp_end))


In [ ]:
print(np.concatenate((chirp_end,np.real(known_ofdm_data),np.real(ofdm_data),np.real(known_ofdm_data))).size)

In [ ]:
plt.plot(output_audio)  #why is there those spikes?
print(known_ofdm_data.size,ofdm_data.size)

In [ ]:
#save and play
audio.signal_to_wav(np.real(output_audio), fs, 'ofdm_tester', 'audio_files')
sd.play(output_audio,fs,blocking=True)


In [ ]:
octagon = channels.octagon(os.path.join(cwd, "audio_files"), fs)
channel = octagon[100:500]
received = channels.channel_output(channel, np.real(output_audio))
audio.signal_to_wav(np.real(received), fs, 'ofdm_tester', 'audio_files')

In [ ]:
# Send through Octagon
octagon = channels.octagon(os.path.join(cwd, "audio_files"), fs)
impulse_channel = np.array([1])
channel = np.ones(10)
H = np.fft.fft(channel)

received = channels.channel_output(channel, np.real(ofdm_data))
fft = ofdm.ofdm_to_fourier(received[:ofdm_data.size], DFT_LENGTH, CP_LENGTH)
discarded = ofdm.subcarrier_extract(fft, DFT_LENGTH, fs, low_freq, high_freq)
deconvolved = ofdm.deconvolve(discarded, H, DFT_LENGTH, fs, low_freq, high_freq)
if encode_method == 'qpsk':
    decoded = decode.qpsk_decode(deconvolved)
elif encode_method == 'bpsk':
    decoded = decode.bpsk_decode(deconvolved)
print(encoded.size, decoded.size)
print("source:\n", encoded[:30])
print("decoded:\n", decoded[:30])



In [ ]:
equality = encoded == decoded[:encoded.size]
equality = equality[10:]
print("correct rate:\n", np.sum(equality.astype(int))/equality.size)